Name:   
Matrikelnummer:  
Email:   
   
Name:   
Matrikelnummer:   
Email:

Name:    
Matrikelnummer:    
Email:    

### Note: This assignment will extensively refer to coding exercise in assignment 4.

## 6.2.a Building your own Neural-Network

Import numpy, which is really all we need to create our own NN.

In [198]:
import numpy as np
# Remember to Fix your seeds for pytorch and numpy

Recall that our simple neural network consisted of two layers. We also added an `activation` function as a non-linearity to the output of our intermediate layer. Given an input $\mathbf{x} \in \mathbb{R}^n $ we have

$ \mathbf{h} = f^{(1)}(\mathbf{x}; \mathbf{W},c) = activation\_fn(\mathbf{W}^\mathsf{T} \mathbf{x} + c) $ \


$ \mathbf{y} = f^{(2)}(\mathbf{h}; \mathbf{w},b) = \text{softmax}( \mathbf{w}^\mathsf{T} \mathbf{h} + b) $

In this exercise you will create your own network and are free to implement it with your own design choices. However, we will do it in a way that allows you to specify the depth of network, i.e. we extend our network such that there isn't just one $\mathbf{h}$ intermediate layers, but rather $n$ of them $\mathbf{h}_{i}$ with $i \in \{1,..., n\}$

**NOTE**: You are not allowed to use any built-in functions to calculate Leaky_ReLU, Softmax or the forward/backward pass directly.

**NOTE 2**: Remember to include the non-linearity at every layer. Remember to also add the bias to every layer. Finally, remember to apply the softmax in the output layer.

## ToDo: Rewrite the Leaky_ReLu and Softmax function as Class and implement a function in each of them to calculate gradients (1 point)
Remember that in PyTorch, these are implemented as classes so we also want to have them as classes.

In [199]:
class leaky_relu:
    """
    Recall your implementation of relu function in assignment 4 and try to implement
    Leaky_ReLu similarily, but as a class with a function to calculate gradient
    """

    def __init__(self, alpha=0.2):
        self.alpha = alpha
        self.output = None

    def forward(self, x):
        x[x<0] = x[x<0] * self.alpha
        self.output = x.copy()  # for backpropagation?
        return x

    def gradient(self):
        assert self.output is not None
        grad = np.ones(self.output.shape[0], dtype=float)
        grad[self.output > 0] = 1.0
        grad[self.output < 1] = self.alpha

        return grad

In [200]:
class softmax:
    """
    Recall your implementation of softmax function in assignment 4 and try to implement
    softmax similarily, but as a class with a function to calculate gradient
    """
    def __init__(self):
        self.output = None
        self.input = None

    def forward(self, x):
        self.input = x.copy()
        x = np.exp(x) / (np.exp(x).sum())
        self.output = x.copy()
        return x

    def gradient(self):
        length = self.output.shape[0]

        grad = np.zeros((length, length))
        for i in range(length):
            for j in range(length):
                if i == j:
                    grad[i][j] = self.output[i] * (1 - self.output[j])
                else:
                    grad[i][j] = self.output[i] * (0 - self.output[j])

        return grad

## ToDo: Calculate the gradient using your implemented functions in their respective classes and validate by manually calculating gradients using a toy value. (1 point)

In [201]:
## Solution
input_array = np.array([1.0, 1, 0, -1])

# relu
relu = leaky_relu()
relu_output = relu.forward(input_array)
print('[Relu] output:\n', relu_output)
# gradient
gradient = relu.gradient()
print('[Relu] gradient:\n', gradient)

# softmax
input_array = np.array([1, 1, 1, -1])
cstm_sftmx = softmax()
softmax_output = cstm_sftmx.forward(input_array)
softmax_gradient = cstm_sftmx.gradient()
print('[Softmax] output:\n', softmax_output)
print('[Softmax] gradient:\n', softmax_gradient)

# is more expected???
# WHat does "validate by manually calculating gradients using a toy value."
# Relu is trivial
# Softmax is an algorithmic implementation of my proof => same result

[Relu] output:
 [ 1.   1.   0.  -0.2]
[Relu] gradient:
 [1.  1.  0.2 0.2]
[Softmax] output:
 [0.31894516 0.31894516 0.31894516 0.04316453]
[Softmax] gradient:
 [[ 0.21721914 -0.10172601 -0.10172601 -0.01376712]
 [-0.10172601  0.21721914 -0.10172601 -0.01376712]
 [-0.10172601 -0.10172601  0.21721914 -0.01376712]
 [-0.01376712 -0.01376712 -0.01376712  0.04130136]]


## ToDo: Rewrite the code from Assignment 4 to include backpropagation in your class without using pytorch. Remember to use your Leaky_ReLu class here as activation function. (1.5 points)
#### Feel free to refer to your solutions from Assignment 4.

In [236]:
class FFNetwork:
    """
    Class representing the feed-forward neural network
    """
    def __init__(self, input_dim: int, hidden_dim: int,
                 output_dim: int, hidden_size: int, lr:float = 0.01):
        """
        Args:
        input_dim: dimensionality of `x`
        hidden_dim: dimensionality of the intermediate `h_i`
        output_dim: dimensionality of `y`
        hidden_size: number of intermediate layers `h_i`
        """

        ## SOLUTION ##

        self.layers = []

        # first layer
        self.layers.append([np.random.randn(input_dim, hidden_dim), np.zeros(hidden_dim), leaky_relu()])

        # middle layer(s)
        for i in range(hidden_size):
            self.layers.append([np.random.randn(hidden_dim, hidden_dim), np.zeros(hidden_dim), leaky_relu()])

        # last layer
        self.layers.append([np.random.randn(hidden_dim, output_dim), np.zeros(output_dim), softmax()])

        self.res = None
        self.inputs = None

        self.layer_output = []

        self.grads = []
        self.lr = lr

        ## SOLUTION ##

    def forward(self, x):
        """
        Args:
        x: input to the neural network

        Output:
        `y`, i.e. the prediction of the network

        Note: Remember to apply the ReLU and add the bias for each layer
        """
        res = x
        self.inputs = x.copy()

        ## SOLUTION ##

        # first layer
        W, b, activation = self.layers[0]
        res = res@W + b
        res = activation.forward(res)

        self.layer_output.append(res.copy())

        # middle layers
        for W, b, activation in self.layers[1:-1]:
            res = res@W + b
            res = activation.forward(res)
            self.layer_output.append(res.copy())

        # last layer
        W, b, activation = self.layers[-1]
        res = res@W + b
        res = activation.forward(res)
        self.layer_output.append(res.copy())

        ## SOLUTION ##
        self.res = res

        return res

    def backpropagation(self):

        # compute the gradients w.r.t. the weights and biases
        for i, (W, b, activation) in enumerate(reversed(self.layers)):
            activation_gradient = np.array(activation.gradient())

            layer_output = np.array(self.layer_output[:-i-1])
            if layer_output.shape[0] == 0:
                layer_output = np.array(self.inputs)
                layer_output = np.expand_dims(layer_output, axis=1).T

            if len(activation_gradient.shape) == 1:
                activation_gradient = np.expand_dims(activation_gradient, axis=1)


            # print('## Layer:', len(self.layers) - i, "##")
            # print('grad:', activation_gradient.shape)
            # print('res:', layer_output.shape)
            # print('W:', W.shape)
            # print('b:', b.shape)

            dW = activation_gradient@layer_output
            db = activation_gradient.T@(np.ones((activation_gradient.shape[0], 1)))

            self.grads.append((dW, db))

        # update the weights
        for i in range(len(self.layers)):
            W, b, a = self.layers[-i-1]
            dW, db = self.grads[-i-1]

            self.layers[-i-1] = (
                W - self.lr * dW,
                b - self.lr * db,
                a
            )






np.random.seed(0)

network = FFNetwork(2, 2, 2, 1)
print(network.forward([1.,0.]))

network.backpropagation()

print(network.grads)
print(network.grads[-1][0])

[0.19458777 0.80541223]
[(array([[-0.29943606,  0.11864969],
       [ 0.29943606, -0.11864969]]), array([[ 0.00000000e+00],
       [-5.55111512e-17]])), (array([[1.76405235, 0.40015721],
       [0.35281047, 0.08003144]]), array([[1.2]])), (array([[1. , 0. ],
       [0.2, 0. ]]), array([[1.2]]))]
[[1.  0. ]
 [0.2 0. ]]


# same layer in pytorch

In [235]:
import torch
from torch.autograd import Variable

np.random.seed(0)

network = FFNetwork(2, 2, 2, 1)
network.forward([1.,0.])

torch_params = []
ppp = []

for W, b, _ in network.layers:
    _W = Variable(torch.tensor(W.copy()), requires_grad=True)
    _b = Variable(torch.tensor(b.copy()), requires_grad=True)

    torch_params.append((_W, _b))

    ppp.append(_W)
    ppp.append(_b)



# optimizer
sgd = torch.optim.SGD(lr=network.lr, params=ppp)
sgd.zero_grad()

# forward
x = torch.tensor([1.,0.], dtype=torch.double)
lrelu = torch.nn.LeakyReLU(0.2)
sftmx = torch.nn.Softmax(dim=0)

W, b = torch_params[0]
x = lrelu(x@W + b)

W, b = torch_params[1]
x = lrelu(x@W + b)

W, b = torch_params[2]
x = sftmx(x@W + b)

# x = torch.sum(x-torch.tensor([0, 1]))

print(x)

x[1].backward()
# sgd.step()

# W, b = torch_params[0]
W.grad



tensor([0.1946, 0.8054], dtype=torch.float64, grad_fn=<SoftmaxBackward0>)


tensor([[-0.5759,  0.5759],
        [ 0.0559, -0.0559]], dtype=torch.float64)

### 6.2.b.2 Training a network for MNIST (1.5 points)

Now that we know how to train a Neural network in Pytorch. Let's start training and evaluating our model using a very standard dataset, for now let's use MNIST. Design a network from scracth using PyTorch and include the followings. Remember that we need to use forward-propagation and backprop.
- Training Loop
- Optimization 
- Evaluating Loop

In [204]:
import torch
import torch.nn.functional as F

from torch import nn

In [205]:
class TorchFFNetwork(nn.Module):
    """
    A pytorch implementation to do classification for MNIST dataset.
    """

    def __init__():
        raise NotImplementedError

### ToDo: Implement functions for Stochastic Gradient Descent and Stochastic Gradient Descent with momentum and plot the difference on how they change the value for gradients. ( 1 + 1 (Bonus))